In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
"""
This function receives a Driver, the initial page adress and the product which you want to search for.
"""

def search_product(CONST_URL, driver,product_name):
    try:
        # getting the input bar and the search button
        search_input = driver.find_element(By.CLASS_NAME, "nav-search-input")
        search_btn = driver.find_element(By.CLASS_NAME, "nav-search-btn")

        # send the product name to the search bar and press the serch button
        search_input.send_keys(product_name)
        search_btn.click()
    except:
        print("Can't search for the product")

In [3]:
"""
This function get all the announcements current page and returns their links.
"""
def get_products_names_and_links(driver):
    announcements = driver.find_element(By.XPATH, "//ol[@class='ui-search-layout ui-search-layout--stack shops__layout']").find_elements(By.TAG_NAME,"li")
    announcements_links = []
    for announcement in announcements:
        try:
            name = announcement.find_element(By.XPATH, ".//a[@class='ui-search-item__group__element ui-search-link__title-card ui-search-link']").get_attribute('text')
            link = announcement.find_element(By.XPATH, ".//a[@class='ui-search-item__group__element ui-search-link__title-card ui-search-link']").get_attribute('href')
            announcements_links.append(link)
        except:
            "error"
    
    return announcements_links


In [4]:
"""
This function receives a product url and returns a dict that contains all the information 
about the product
"""
def get_product_information(product_url, driver):
    driver.get(product_url)
    product_info = {}

    # get announcement name and link
    name = driver.find_element(By.XPATH, "//h1[@class='ui-pdp-title']").get_attribute("innerHTML")
    link = product_url
    product_info["name"] = name
    product_info["link"] = link
            
    #### get prices 
    price = driver.find_element(By.XPATH, "//div[@class='ui-pdp-price__second-line']")
    price = price.find_element(By.XPATH, ".//span[@class='andes-money-amount__fraction']").get_attribute("innerHTML")
    product_info["price"] = price

    # get tables
    tables = driver.find_elements(By.XPATH, "//table[@class='andes-table']")

    # for each table, getting all the information inside it
    for table in tables:
        # getting all rows of the table
        rows = table.find_elements(By.XPATH, ".//tr[@class='andes-table__row ui-vpp-striped-specs__row']")
        
        # for each row, get column name and the content inside it.
        for row in rows:
            column_name = row.find_element(By.XPATH, ".//div[@class='andes-table__header__container']").get_attribute("innerHTML")
            column_content = row.find_element(By.XPATH, ".//span[@class='andes-table__column--value']").get_attribute("innerHTML")
            product_info[column_name] = column_content
        
    return product_info


In [5]:
"""
This function catchs the button that redirects to the next pages.
Then, it redirects the driver to the next page.
"""
def go_next_page(driver):
    # get next page button
    try:
        next_btn = driver.find_element(By.XPATH, "//li[@class='andes-pagination__button andes-pagination__button--next']")
        next_page_link = next_btn.find_element(By.XPATH, ".//a[@class='andes-pagination__link']").get_attribute("href")
        driver.get(next_page_link)
        return 1
    except:
        return 0

In [11]:
"""
This function compiles all the other functions and define a workflow to get products information.
It receives 'number_products_to_collect' that defines how many announcements the bot should collect
and 'product' that define the product in which the bot should search for.
The funcion returns a dataframe in which each row represents a announcement.
"""
def get_products(CONST_URL, driver, product, number_products_to_collect):
    products_info = []

    #go to the initial page
    driver.get(CONST_URL)

    # search for the product
    search_product(CONST_URL, driver, product)

    #getting the product url (initially the first page of the product)
    product_url_comeback = driver.current_url

    continue_passing, products_collected = 1, 0

    while(continue_passing and products_collected < number_products_to_collect):
        # in the current page, get all the annoucements (names and their links)
        annoucements_links = get_products_names_and_links(driver)

        # for every announcement link, get informations about the product
        for annoucement_link in annoucements_links:
            #if we have already collected enough products, go out.
            if products_collected >= number_products_to_collect:
                break

            product_info = get_product_information(annoucement_link, driver)
            products_info.append(product_info)
            products_collected += 1
            time.sleep(1)

        #comeback to the page with the announcements
        driver.get(product_url_comeback)

        # go to the next page
        continue_passing = go_next_page(driver)
        product_url_comeback = driver.current_url
        print(f"have gone to the next page: {product_url_comeback}")
        
    return pd.DataFrame(products_info)  
    

### User Case Example 

In [12]:
driver = Firefox()
CONST_ML_URL = "https://www.mercadolivre.com.br/"


products_df = get_products(CONST_ML_URL, driver, "creatina", 50)

have gone to the next page: https://lista.mercadolivre.com.br/saude/suplementos-alimentares/creatina_Desde_51_NoIndex_True


In [13]:
products_df

,name,link,price,Marca,Linha,Suplemento principal,Nome do suplemento,Formato do suplemento,Tipo de embalagem,Sabor,Formato de venda,Peso líquido,Volume líquido,Volume da unidade,Peso da unidade,Ingredientes,Valores nutricionais por porção,Peso da porção,Volume da porção,Aminoácidos por porção,Idade mínima recomendada,Uso recomendado,É livre de glúten,Contém lactose,É vegano,Quantidade de porções,É apto para as pessoas durante a gravidez,É orgânico,Componentes alergéneos,É apto para as pessoas lactantes,Unidades por kit,Unidades por embalagem
0,Creatina Monohidratada Pura 500g Dark Lab Unidade,https://click1.mercadolivre.com.br/mclics/clic...,78,Dark Lab,Premium,Creatina,Pure Creatine,Pó,Sachê,Sem sabor,Unidade,500 g,500 mL,500 mL,500 g,Creatina monohidratada,Tabela nutricional,3 g,3 mL,tabela nutricional,18 anos,Nutricional/Esportivo,Sim,Não,Sim,166,Não,Não,"Não contem gluten. Pode conter soja, Ovo, amen...",NaN,NaN,NaN
1,Creatina Monohidratada Pura 1kg - Dark Lab Uni...,https://click1.mercadolivre.com.br/mclics/clic...,166,Dark Lab,Premium,Creatina,Pure Creatine,Pó,Sachê,Sem sabor,Unidade,1 kg,1000 mL,1000 mL,1 kg,Creatina monohidratada,3000 mg creatina monohidratada,3 g,3 mL,3000 mg creatina monohidratada,18 anos,Nutricional/Esportivo,Sim,Não,Sim,333,Não,Não,"Não contem gluten. Pode conter soja, Ovo, amen...",NaN,NaN,NaN
2,Suplemento em Pó Turbo 300g Black Skull Sem s...,https://www.mercadolivre.com.br/suplemento-em-...,29,Black Skull,Preta Caveira,Carboidratos,Creatine Turbo,Pó,Pote,Sem sabor,Unidade,300 g,300 mL,300 mL,300 g,"Maltodextrina, Creatina monohidratada","12 kcal valor energético, 3 g carbohidratos, 3...",6 g,150 mL,"Maltodextrina, creatina",15 anos,Nutricional/Esportivo,Sim,Não,Não,50,Não,Não,"Não Contém Gluten, Não Contém Lactose",Não,NaN,NaN
3,Creatina 150g - Dark Lab Sabor Sem Sabor,https://click1.mercadolivre.com.br/mclics/clic...,39,Dark Lab,Premium,Creatina,Pure Creatine,Pó,Pote,Sem sabor,Unidade,150 g,150 mL,150 mL,150 g,Creatina monohidratada,Tabela nutricional,3 g,3 mL,tabela nutricional,18 anos,Nutricional/Esportivo,Sim,Não,Sim,50,Não,Não,"Não contem gluten. Pode conter soja, Ovo, amen...",NaN,NaN,NaN
4,Suplemento em pó Espartanos Espartanos Creati...,https://click1.mercadolivre.com.br/mclics/clic...,159,Espartanos,Creatina Pura,Creatina 100% Integral,Creatine Max ima Absorção,Pó,Pote,Sem sabor,Unidade,1000 g,40 mL,1000 mL,1000 g,"Creatina monohidratad, Creatina pura",3 g de Creatina,3 g,3 mL,3 g de Creatina,12 anos,Nutricional/Esportivo,Sim,Não,Não,NaN,Não,Não,NaN,Não,NaN,NaN
5,Suplemento em Pó Creatine Turbo Sabor Limão 30...,https://www.mercadolivre.com.br/suplemento-em-...,29,Black Skull,Preta Caveira,Carboidratos,Creatine Turbo,Pó,Pote,Limão,Unidade,300 g,300 mL,300 mL,300 g,"Maltodextrina, Creatina monohidratada","12 kcal valor energético, 3 g carbohidratos",6 g,150 mL,"Maltodextrina, creatina",15 anos,Nutricional/Esportivo,Sim,Não,Não,50,Não,Não,"Não Contém Gluten, Não Contém Lactose",Não,NaN,NaN
6,Pack 2 Suplementos em Pó Universal Creatine Mo...,https://www.mercadolivre.com.br/pack-2-supleme...,113,Universal Nutrition,Classic Series,Creatina monohidratada,Creatina,Pó,Lata,Sem sabor,Kit,200 g,200 mL,200 mL,200 g,Creatina,5 g creatina monohidratada,3 g,3 mL,Creatina - 3g,18 anos,Esportivo,Não,Sim,Não,40,NaN,NaN,Derivados de leche,NaN,2,NaN
7,Suplemento em pó Creatine Turbo Caveira Preta ...,https://www.mercadolivre.com.br/suplemento-em-...,46,Black Skull,Preta Caveira,Carboidratos,Creatine Turbo,Pó,Sachê,Sem sabor,Unidade,500 g,500 mL,500 mL,500 g,"Maltodextrina, Creatina monohidratada","12 kcal valor energético, 3 g carbohidratos, 3...",6 g,150 mL,"Maltodextrina, creatina",15 anos,Nutricional/Esportivo,Sim,Não,Não,83,Não,Não,"Não Contém Gluten, Não Contém Lactose",Não,NaN,NaN
8,Creatina Monohidratada 500g Suplemento em pó 1...,https://click1.mercadolivre.com.br/mclics/clic...,77,Espartanos,Creatina Pura,Creatina 100% Integral,Creatine Max ima Absorção,Pó,Pote,Sem sabor,Unidade,500 g,500 mL,NaN,